# Offset for overlapping bus routes


The bus routes, along with the bus stops, will create using QGIS software utilising OSM road network (copying out individual sections): a LineString layer for the routes and a Point layer for the bus stops. After processing the data with Pandas and Geopandas will export as GeoJSON to use with Leaflet for a web map. Currently Leaflet lacks a method to offset a line (Polyline), so will use the Leaflet Polyline Offset plugin: https://github.com/bbecquet/Leaflet.PolylineOffset

As these routes are created from OSM road network each is already divided into segments, otherwise at least the overlapping segments need to be separated. Once loaded into here, a list of routes will be generated for each segment, listing one or multiple bus lines using the same segment. The offset plugin then iterates through the route list of each segment to set the offset value for each bus route in the list. For instance, if a segment is used only in one bus route there will be no need to offset it, however if a segment is used in multiple routes, each will be slightly offset. Also a column containing the offset values will be created be able to offset routes within QGIS layer styling.
<br><br>
This all done without altering the geometries of the routes.

In [1]:
import pandas as pd
import geopandas

### Bus Stops

*loading only to show the data*

In [3]:
stops = geopandas.read_file('data_routes/bus_stops.geojson')
stops.head()

,name,geometry
0,sa,POINT (24.85339 57.15374)
1,su,POINT (24.84468 57.15712)
2,tu,POINT (24.84887 57.16121)
3,ta,POINT (24.85603 57.16195)
4,pa,POINT (24.86309 57.16642)


### Bus Routes
*the route data consists of bus route numbers and geometry*

In [4]:
gdf = geopandas.read_file('data_routes/routes_to_process.geojson')
gdf.head(3)

,route,geometry
0,1,"LINESTRING (24.85092 57.15608, 24.85097 57.156..."
1,1,"LINESTRING (24.85497 57.16187, 24.85505 57.161..."
2,1,"LINESTRING (24.85124 57.16146, 24.85104 57.161..."


In [5]:
# creating route list for each segment
# grouping by geometry and aggregating route numbers into a list, reseting index to have a dataframe
grouped = gdf.groupby('geometry')['route'].apply(list).reset_index()
grouped = grouped.rename(columns={'route': 'route_list'})


# adding the route list column to the main dataframe
gdf = gdf.merge(grouped, on='geometry', how='left')


# converting to string type (needed in order to export to a file)
gdf['route_list'] = gdf['route_list'].astype(str)
gdf.head(3)

,route,geometry,route_list
0,1,"LINESTRING (24.85092 57.15608, 24.85097 57.156...","[1, 3]"
1,1,"LINESTRING (24.85497 57.16187, 24.85505 57.161...","[1, 2, 3, 5]"
2,1,"LINESTRING (24.85124 57.16146, 24.85104 57.161...",[1]


In [6]:
# this step is for offsetting routes in QGIS software (via Layer Styling)
# and is not needed for Leaflet web map


# will create a column with offset values

gdf['offset'] = 0


# a new dataframe for each route
r1 = gdf.loc[gdf['route'] == 1]
r2 = gdf.loc[gdf['route'] == 2]
r3 = gdf.loc[gdf['route'] == 3]
r4 = gdf.loc[gdf['route'] == 4]
r5 = gdf.loc[gdf['route'] == 5]


# route 1: offset stays 0


# route 2
merged = r2.reset_index().merge(r1, how='inner', on='geometry').set_index('index')
merged['offset'] = merged.groupby(merged['geometry'].to_wkt())['geometry'].transform('count')
r2.update(merged.loc[~merged.duplicated(subset=['geometry']), 'offset'])


# route 3
# r3 inner joined with r2 + r1 on 'geometry' and r3 index is kept
merged = r3.reset_index().merge(pd.concat([r2, r1]), how='inner', on='geometry').set_index('index')
# same name column (as in r3) is created for offset values
merged['offset'] = merged.groupby(merged['geometry'].to_wkt())['geometry'].transform('count')
# r3 is updated with these offset values based on index
r3.update(merged.loc[~merged.duplicated(subset=['geometry']), 'offset'])


# route 4
merged = r4.reset_index().merge(pd.concat([r3, r2, r1]), how='inner', on='geometry').set_index('index')
merged['offset'] = merged.groupby(merged['geometry'].to_wkt())['geometry'].transform('count')
r4.update(merged.loc[~merged.duplicated(subset=['geometry']), 'offset'])


# route 5
merged = r5.reset_index().merge(pd.concat([r4, r3, r2, r1]), how='inner', on='geometry').set_index('index')
merged['offset'] = merged.groupby(merged['geometry'].to_wkt())['geometry'].transform('count')
r5.update(merged.loc[~merged.duplicated(subset=['geometry']), 'offset'])


gdf = pd.concat([r1, r2, r3, r4, r5])
gdf

,route,geometry,route_list,offset
0,1,"LINESTRING (24.85092 57.15608, 24.85097 57.156...","[1, 3]",0
1,1,"LINESTRING (24.85497 57.16187, 24.85505 57.161...","[1, 2, 3, 5]",0
2,1,"LINESTRING (24.85124 57.16146, 24.85104 57.161...",[1],0
3,1,"LINESTRING (24.84898 57.16156, 24.84896 57.161...",[1],0
4,1,"LINESTRING (24.85542 57.15418, 24.85632 57.154...","[1, 3, 4]",0
...,...,...,...,...
170,5,"LINESTRING (24.86408 57.14678, 24.86392 57.146...","[2, 4, 5]",2
171,5,"LINESTRING (24.85337 57.14453, 24.85237 57.14432)","[4, 5]",1
172,5,"LINESTRING (24.85237 57.14432, 24.84914 57.143...","[4, 5]",1
173,5,"LINESTRING (24.84371 57.14386, 24.84370 57.14395)",[5],0


### Exporting the data for some more processing in QGIS:
*(not found an easy way of doing it here)*

In [8]:
gdf.to_file('data_routes/to_dissolve.gpkg', index=False, driver="GPKG")

Using QGIS Processing Tools:
1. 'Dissolve' on all the fields ('route', 'route_list', and 'offset')
2. 'Multipart to Singleparts'

The result should be a LineString layer which then needs to be exported as a GeoJSON file.

Both GeoJSON files (routes and stops) are used with Leaflet map, a working demo: https://map.sigulda.lv/routes.html